In [1]:
import pandas as pd

In [90]:
from itertools import product

hpi_method = ['hypershap', 'fanova', 'random']
adjust_cs = ['true', 'false']
adjust_previous_cfgs = ['true', 'false']
dynamic_decay = ['linear', 'none']

adjust_cs_method = ['default', 'random', 'incumbent']
constant = ['true', 'false']
cs_proba_hpi = ['true', 'false']

set_to = ['random', 'default', 'incumbent']

thresh_list = ['[0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3]', '[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]']
thresh = [0.5, 0.75]

# Generate all combinations
combinations = list(product(
    hpi_method,
    adjust_cs,
    adjust_previous_cfgs,
    dynamic_decay,
    adjust_cs_method,
    constant,
    cs_proba_hpi,
    set_to,
    thresh_list,
    thresh
))

columns = [
    'hpi_method', 'adjust_cs', 'adjust_previous_cfgs', 'dynamic_decay',
    'adjust_cs_method', 'constant', 'cs_proba_hpi', 'set_to',
    'thresh_list', 'thresh'
]
df = pd.DataFrame(combinations, columns=columns)

In [ ]:

# # Conditional config
# conditional_configs:
#   adjust_cs_specific:
#     enabled: ${adjust_cs} == "true"
#     adjust_cs_method: [default, random, incumbent]
#     constant: [true, false]
#     cs_proba_hpi: [true,false]
#     # if cs_proba==true, consnt==false
#   adjust_previous_specific:
#     enabled: ${adjust_previous_cfgs} == "true"
#     set_to: [random, default, incumbent]
#   decay_specific:
#     enabled: ${dynamic_decay} == "linear"
#     thresh_list: [[0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3], [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]]
#   no_decay_specific:
#     enabled: ${dynamic_decay} == "none"
#     thresh: [0.5,0.75]

In [91]:
def remove(row, li):
    for col in li:
        row[col] = None
    return row

df[df['adjust_cs']=='false'] = df[df['adjust_cs']=='false'].apply(lambda row: remove(row, ['adjust_cs_method', 'constant', 'cs_proba_hpi']), axis=1)
df[(df['adjust_cs']=='true')&(df['cs_proba_hpi']=='true')] = df[(df['adjust_cs']=='true')&(df['cs_proba_hpi']=='true')].apply(lambda row: remove(row, ['constant']), axis=1)
df[(df['adjust_previous_cfgs']=='false')] = df[(df['adjust_previous_cfgs']=='false')].apply(lambda row: remove(row, ['set_to']), axis=1)
df[(df['dynamic_decay']=='none')] = df[(df['dynamic_decay']=='none')].apply(lambda row: remove(row, ['thresh_list']), axis=1)
df[(df['dynamic_decay']=='linear')] = df[(df['dynamic_decay']=='linear')].apply(lambda row: remove(row, ['thresh']), axis=1)

/tmp/ipykernel_24917/3269138565.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[None None None ... None None None]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df[(df['dynamic_decay']=='linear')] = df[(df['dynamic_decay']=='linear')].apply(lambda row: remove(row, ['thresh']), axis=1)


In [92]:
df = df.drop_duplicates()
df.shape

(480, 10)

In [94]:
df.columns

Index(['hpi_method', 'adjust_cs', 'adjust_previous_cfgs', 'dynamic_decay',
       'adjust_cs_method', 'constant', 'cs_proba_hpi', 'set_to', 'thresh_list',
       'thresh'],
      dtype='object')

In [97]:
commands = []

for _, row in df.iterrows():
    # Construct the command dynamically, skipping None values
    command = "sbatch start_hydra.sh"
    baserundir = f"result"
    
    command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.hpi_method={row['hpi_method']}"
    baserundir += f"_{row['hpi_method']}"
    
    command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.adjust_cs={row['adjust_cs']}"
    if row['adjust_cs'] == 'true':
        baserundir += f"_adjust_cs"
    
    if row['constant'] is not None:
        command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.constant={row['constant']}"
        if row['constant'] == 'true':
            baserundir += f"_{row['constant']}"
    if row['cs_proba_hpi'] is not None:
        command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.cs_proba_hpi={row['cs_proba_hpi']}"
        if row['cs_proba_hpi'] == 'true':    
            baserundir += f"_{row['cs_proba_hpi']}"
    
    command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.adjust_previous_cfgs={row['adjust_previous_cfgs']}"
    if row['adjust_previous_cfgs'] == 'true': 
        baserundir += f"_{row['adjust_previous_cfgs']}"
    
    if row['set_to'] is not None:
        command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.set_to={row['set_to']}"
        baserundir += f"_set_to_{row['set_to']}"
    
    command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.dynamic_decay={row['dynamic_decay']}"
    if row['dynamic_decay'] == 'linear': 
        baserundir += f"_dynamic_decay"
    
    if row['thresh'] is not None:
        command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.thresh={row['thresh']}"
        baserundir += f"_thresh_{row['thresh']}"
        
    if row['thresh_list'] is not None:
        command += f" optimizer.smac_cfg.smac_kwargs.acquisition_maximizer.cs_proba_hpi={row['thresh_list']}"
        if row['thresh_list'] == '[0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3]': 
            baserundir += f"_down"
        elif row['thresh_list'] == '[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]':
            baserundir += f"_up"
        
        
    command += f" baserundir={baserundir}"
    
    commands.append(command)

In [ ]:
import subprocess

# Iterate over the commands and execute them
for command in commands:
    process = subprocess.run(command, shell=True)
    if process.returncode != 0:
        print(f"Command failed: {command}")